# 1. Carregamento dos dados

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
import sklearn
import math
from azureml.core import Experiment

# O azureml-core da versão 1.0.72 ou superior é requerido
# é necessário azureml-dataprep[pandas] na versão 1.1.34 ou superior
from azureml.core import Workspace, Dataset

Autentica na sua conta Azure e acessa o Workspace de ML!

In [ ]:
#Trocar os códigos abaixo pelos da sua instância!
subscription_id = '5298dbd8-b7e6-432a-b8fc-828328490c29'
resource_group = 'Aulas-FIAP'
workspace_name = 'WorkspaceMLAula'

workspace = Workspace(subscription_id, resource_group, workspace_name)

Obtém o Dataset que você subiu nesta instância!

In [ ]:
dataset = Dataset.get_by_name(workspace, name='Risco-de-Credito')
df = dataset.to_pandas_dataframe()
df.head()

# 2. Treinamento

## 2.1 - Separa dados em treino e teste

In [ ]:
x_df = df.loc[:,["renda", "idade", "etnia", "sexo", "casapropria", "outrasrendas", "estadocivil", "escolaridade"] ]
y_df = df.loc[:,"default"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=66)

## 2.2 Random Forest 

Prepara ambiente na nuvem para logar os experimentos

In [ ]:
experiment = Experiment(workspace=workspace, name="risco-credito-experimentos-RF")
run = experiment.start_logging()
run.log("Valor Parametro", 1.0)

Treina (treinamento simples)

In [ ]:
clf = rfc()
clf.fit(X=X_train, y=y_train)
clf.independentcols = x_df.columns

In [ ]:
clf_acuracia = clf.score(X=x_df, y=y_df)
print("Modelo 01 (classificador), criado com acurácia de: [{0}]".format(clf_acuracia))
run.log("acuracia", clf_acuracia)

Demais logs a se registrar para este experimento.

In [ ]:
run.log("Versao sklearn", sklearn.__version__)
run.log("criterion", clf.criterion)
run.log("n_estimators", clf.n_estimators)
run.log("min_samples_leaf", clf.min_samples_leaf)
run.log("max_depth", clf.max_depth)
run.log_list("Inputs", list(x_df.columns) )

Salva modelo e encerra o experimento (Logger)

In [ ]:
model_name = "model_risco_RF01.pkl"
filename = "outputs/" + model_name

joblib.dump(value=clf, filename=filename)
run.upload_file(name=model_name, path_or_stream=filename)
run.complete()

## 2.3 Linerar Model Ridge

Prepara ambiente na nuvem para logar os experimentos

In [ ]:
experiment = Experiment(workspace=workspace, name="risco-credito-experimentos-Ridge")

Faz várias execuções, com pequenas modificações.

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for alpha in alphas:
    print("alpha_value: {0}".format(alpha ) )
    run = experiment.start_logging()
    run.log("alpha_value", alpha)

    model = Ridge(alpha=alpha)
    model.fit(X=X_train, y=y_train)
    model.independentcols = x_df.columns
    y_pred = model.predict(X=X_test)
    acuracia = model.score(X=x_df, y=y_df)
    run.log("acuracia", acuracia)
    run.log("Versao sklearn", sklearn.__version__)
    rmse = math.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred))
    run.log("rmse", rmse)

    model_name = "model_alpha_" + str(alpha) + ".pkl"
    filename = "outputs/" + model_name

    joblib.dump(value=model, filename=filename)
    run.upload_file(name=model_name, path_or_stream=filename)
    run.complete()

-----

# FIM